<!-- -*- mode: markdown; coding: utf-8; fill-column: 60; ispell-dictionary: "english" -*- -->

<meta charset="utf-8"/>
<meta name="viewport" content="width=device-width,initial-scale=1"/>
<link rel="stylesheet" href="../style.css">

# More SQL (lecture 2)

Today we'll continue where we left off last time, and then
learn how to combine several tables in one `SELECT`
statement.

As usual we have to tell Jupyter to allow SQL:

In [1]:
%load_ext sql

And then we import the database for today's lecture:

In [2]:
%sql sqlite:///lect02.sqlite

'Connected: @lect02.sqlite'

## Short review of the first lecture

 +  `SELECT`-`FROM`-`WHERE`: 
 
 +  Set operations: `UNION`, `INTERSECT`, `EXCEPT`
 
 +  Simple functions: applies to each row of the result,
    like (you can find their documentation
    [here](https://www.sqlite.org/lang_corefunc.html)):
     +  `substr(s, x, y)`
     +  `instr`/`glob`
     +  `ltrim`/`rtrim`/`trim`
     +  `ifnull`/`coalesce`
     +  `printf`
     +  `abs`/`round`
     +  `max(x,y,...)`/`min(x,y,...)` -- observe that this
        is not the same as the aggregate functions (see
        below)
     +  `random`/`randomblob`
     +  `hex`
     +  `lower`/`upper`

 +  Aggregate functions (documented
    [here](https://www.sqlite.org/lang_aggfunc.html)):
     +  `avg`
     +  `count`
     +  `group_concat`
     +  `max`/`min`
     +  `sum`/`total`
 
 +  `GROUP BY` -- splits the rows into groups, and applies
    aggregate functions on each group (instead of on the
    whole table)
    
 +  `GROUP BY` ... `HAVING` -- like `WHERE` for groups
 
 +  Subqueries -- we can use queries within other queries,
    typically in a `WHERE ... IN`-statement


## Subqueries, continued...

At the end of the previous lecture, we looked at the
(admittedly contrieved) question:

**Q:** _Who has shared the chemistry prize with exactly one
other laureate in years when the summer olympics were held
in Europe?_

In [ ]:
%%sql


We'll save the following problem for the QA-session:

**Q:** _Print the names of all the physics and chemistry
laureates in years when both categories had laureates with
names beginning with 'A'?_

In [29]:
%%sql
SELECT year, group_concat(name, ', ') AS names, group_concat(category,', ') AS categories
FROM nobel
WHERE name IN
    (SELECT name
     FROM nobel
     WHERE (category='chemistry' OR category='physics') AND name LIKE 'A%'
    ) 
GROUP BY year HAVING count() > 1

 * sqlite:///lect02.sqlite
Done.


year,names,categories
2000,"Alan G. MacDiarmid, Alan J. Heeger","chemistry, chemistry"
2003,"Alexei A. Abrikosov, Anthony J. Leggett","physics, physics"
2004,"Aaron Ciechanover, Avram Hershko","chemistry, chemistry"
2010,"Akira Suzuki, Andre Geim","chemistry, physics"
2015,"Aziz Sancar, Arthur B. McDonald","chemistry, physics"


Another form of subquery is:

~~~sql
SELECT ...,
       (SELECT ...
        FROM ...
        WHERE ...)
FROM   ...
~~~


This works if the subquery produces one result, such as when
we use an aggregate function. As an example, solve the
following problem:

**Q:** _Output the name of all laureates, and the number of
awards they have -- order first by number of awards, then by
name, and show only the first 20._

In [11]:
%%sql
SELECT name, (SELECT count()
             FROM nobel AS inner
             WHERE inner.name = outer.name) AS no_of_awards
FROM nobel AS outer
GROUP BY name
ORDER BY no_of_awards DESC, name


 * sqlite:///lect02.sqlite
Done.


name,no_of_awards
Frederick Sanger,2
John Bardeen,2
"Marie Curie, née Sklodowska",2
Aage Niels Bohr,1
Aaron Ciechanover,1
Aaron Klug,1
Abdus Salam,1
Ada E. Yonath,1
Adam G. Riess,1
Adolf Friedrich Johann Butenandt,1


This is called a _correlated subquery_ (since we refer to
the enclosing query inside it). We use an alias to
distinguish between the nobel table in the outer query and
the nobel table in the subquery (it's the same table, but we
'iterate' through it separately).



## Redundancy, and the case for splitting up tables

The term
[redundancy](https://en.wikipedia.org/wiki/Redundancy_(linguistics))
can be defined in many contexts, in linguistics (and in
general) it refers to information which is expressed more
than once. It is sometimes desirable (the human body has an
enormous amount of redundancy, and that's what enable us to
self-heal), but often it is a source of confusion and
errors.

Our table of Olympic Games looks like this:

In [30]:
%%sql
SELECT *
FROM   olympics

 * sqlite:///lect02.sqlite
Done.


year,city,country,continent,season,ordinal_number
1924,Chamonix,France,Europe,winter,I
1928,St. Moritz,Switzerland,Europe,winter,II
1932,Lake Placid,United States,North America,winter,III
1936,Garmisch-Partenkirchen,Germany,Europe,winter,IV
1948,St. Moritz,Switzerland,Europe,winter,V
1952,Oslo,Norway,Europe,winter,VI
1956,Cortina d'Ampezzo,Italy,Europe,winter,VII
1960,Squaw Valley,United States,North America,winter,VIII
1964,Innsbruck,Austria,Europe,winter,IX
1968,Grenoble,France,Europe,winter,X


**Exercise:** _Can you see any potential sources of
redundancy?_

<p>&nbsp;</p>
<p>&nbsp;</p>
<p>&nbsp;</p>
<p>&nbsp;</p>
<p>&nbsp;</p>
<p>&nbsp;</p>

Two things come to mind:

 * A city seldom changes country (although it happens...),
   so repeating the same pairs of (city, country) in several
   places may seem redundant.

 * A country never moves to another continent, so repeating
   the same pairs of (country, continent) is always
   redundant.
 
In the first case, there might be a reason to save both the
city name and the country name, not only because counties
historically have changed (in 1980 Moscow was in the Soviet
Union, and in 1984 Sarajevo was in Yugoslavia), but also
because city names aren't globally unique -- there is a
Paris in Texas, and a Moscow in Kansas. So we may need the
country to identify the city uniquely. In this case we say
that the pair (city, country) is a _key_ (we'll return to
this many times during the course).

But saving the same pair of country and continent in several
places is just redundant, and it is not only wasteful
memory-wise, but could also lead to some problems when we
modify our table (we'll return to this when we talk about
_normalization_).

We say that the continent is _functionally dependent_ on the
country (or that there is a _functional dependency_ from
countries to continents), and we can avoid the redundancy it
gives rise to by creating a new table, which tells on which
continent each country is. We'll return to the details of
the SQL statements below later in the course, you're not
supposed to understand it right now, but if we evaluate the
cell below we'll get two tables, `better_olympics` and
`continents` who could replace our old `olympics`:

In [13]:
%%sql
DROP TABLE IF EXISTS continents;
CREATE TABLE continents (
  country   TEXT,
  continent TEXT NOT NULL,
  PRIMARY KEY (country)
);

INSERT OR IGNORE
INTO   continents(country, continent)
SELECT country, continent
FROM   olympics;

DROP TABLE IF EXISTS better_olympics;
CREATE TABLE  better_olympics (
  year            INT,
  city            TEXT,
  country         TEXT,
  season          TEXT,
  ordinal_number  TEXT,
  PRIMARY KEY (year, season)
);

INSERT
INTO   better_olympics(year, city, country, season, ordinal_number)
SELECT year, city, country, season, ordinal_number
FROM   olympics;

 * sqlite:///lect02.sqlite
Done.
Done.
26 rows affected.
Done.
Done.
52 rows affected.


[]

Now we don't have to enter the continent the next time
Athens, Paris or London is awarded the games.

Our two tables look like this:

In [14]:
%%sql
SELECT *
FROM   better_olympics;

 * sqlite:///lect02.sqlite
Done.


year,city,country,season,ordinal_number
1924,Chamonix,France,winter,I
1928,St. Moritz,Switzerland,winter,II
1932,Lake Placid,United States,winter,III
1936,Garmisch-Partenkirchen,Germany,winter,IV
1948,St. Moritz,Switzerland,winter,V
1952,Oslo,Norway,winter,VI
1956,Cortina d'Ampezzo,Italy,winter,VII
1960,Squaw Valley,United States,winter,VIII
1964,Innsbruck,Austria,winter,IX
1968,Grenoble,France,winter,X


In [15]:
%%sql
SELECT *
FROM   continents;

 * sqlite:///lect02.sqlite
Done.


country,continent
France,Europe
Switzerland,Europe
United States,North America
Germany,Europe
Switzerland,Europe
Norway,Europe
Italy,Europe
Austria,Europe
Japan,Asia
Yugoslavia,Europe


We've avoided some redundancy, but now we need a way to
combine the information in our two tables, and that's what
_joins_ are for...


## Joining tables together

We want a database for handling college applications, a
number of students applies for various majors at different
colleges. We want to keep track of:

 + student id (similar to stil-id)
 + student name
 + student's grade average (gpa)
 + the size of the student's highschool
 + the names of the colleges
 + the state for each college
 + the enrollment for each college
 + the major applied for
 + the decision ('Y' for accepted, 'N' otherwise)

One way of doing this would be to use one big table:

In [3]:
%%sql
SELECT *
FROM   big_college

 * sqlite:///lect02.sqlite
Done.


s_id,s_name,gpa,size_hs,c_name,state,enrollment,major,decision
123,Amy,3.9,1000,Berkeley,CA,36000,CS,Y
123,Amy,3.9,1000,Cornell,NY,21000,EE,Y
123,Amy,3.9,1000,Stanford,CA,15000,CS,Y
123,Amy,3.9,1000,Stanford,CA,15000,EE,N
234,Bob,3.6,1500,Berkeley,CA,36000,biology,N
345,Craig,3.5,500,Cornell,NY,21000,CS,Y
345,Craig,3.5,500,Cornell,NY,21000,EE,N
345,Craig,3.5,500,Cornell,NY,21000,bioengineering,N
345,Craig,3.5,500,MIT,MA,10000,bioengineering,Y
456,Doris,3.9,1000,None,None,None,None,None


But, just as with our olympics database above, this would
give us a lot of redundance, so we'd like to split this big
table into several related tables.

In lectures 7 and 8 we'll discuss the theory and practice of
splitting up tables -- for now, suggest tables which you
think would make working with our data easier.

<p>&nbsp;</p>
<p>&nbsp;</p>
<p>&nbsp;</p>
<p>&nbsp;</p>
<p>&nbsp;</p>
<p>&nbsp;</p>

We have one table for the students:

In [4]:
%%sql
SELECT *
FROM   students
LIMIT 4

 * sqlite:///lect02.sqlite
Done.


s_id,s_name,gpa,size_hs
123,Amy,3.9,1000
234,Bob,3.6,1500
345,Craig,3.5,500
456,Doris,3.9,1000


One for the colleges:

In [5]:
%%sql
SELECT *
FROM   colleges
LIMIT 4

 * sqlite:///lect02.sqlite
Done.


c_name,state,enrollment
Stanford,CA,15000
Berkeley,CA,36000
MIT,MA,10000
Cornell,NY,21000


And one for all collage applications made by the students:

In [6]:
%%sql
SELECT *
FROM   applications
LIMIT 4

 * sqlite:///lect02.sqlite
Done.


s_id,c_name,major,decision
123,Stanford,CS,Y
123,Stanford,EE,N
123,Berkeley,CS,Y
123,Cornell,EE,Y


The `applications` table uses `s_id` instead of `s_name`,
since we could have several students with the same name (we
actually have two 'Amy') -- `s_id` is guaranteed to be
unique (we'll talk _much_ more about uniqueness next week).

Now assume we want to display all applications, _with the
names of the students_ (not just student id). We then need a
way to combine information in the `applications` table with
information in the `students` table, and we'll do it with a
_join_.


The SQL standard specifies five different kinds of joins:

 + _cross join_
 + _inner join_
 + _left outer join_
 + _right outer join_
 + _full outer join_

The most primitive of these is the cross join -- we can
write it as:

~~~
SELECT      *
FROM        a 
CROSS JOIN  b
~~~


Doing this, we get all combinations of rows from each of
table `a` and `b` -- the rows will contain all attributes
from both tables (unless we make a projection).

If we do it with our `applications` and `students` tables we
get:

In [8]:
%%sql
SELECT      *
FROM        applications 
CROSS JOIN  students
LIMIT 5

 * sqlite:///lect02.sqlite
Done.


s_id,c_name,major,decision,s_id_1,s_name,gpa,size_hs
123,Stanford,CS,Y,123,Amy,3.9,1000
123,Stanford,CS,Y,234,Bob,3.6,1500
123,Stanford,CS,Y,345,Craig,3.5,500
123,Stanford,CS,Y,456,Doris,3.9,1000
123,Stanford,CS,Y,543,Craig,3.4,2000


The number of rows in this table is the product of the
number of rows in each table, and most of the rows in the
combined table are totally uninteresting, since the `s_id`'s
doesn't have anything to do with each other (e.g., in one
line the fact that a student with `s_id` 123 has applied to
CS at Stanford is combined with the fact that there is a
student Bob with `s_id` 234 who has a `gpa` of 3.6).

The only rows of interest to us are those where the student
id from the `applications` table is the same as the student
id from the `students` table, and we can express that as:

In [9]:
%%sql
SELECT      *
FROM        applications 
CROSS JOIN  students
WHERE       applications.s_id = students.s_id

 * sqlite:///lect02.sqlite
Done.


s_id,c_name,major,decision,s_id_1,s_name,gpa,size_hs
123,Stanford,CS,Y,123,Amy,3.9,1000
123,Stanford,EE,N,123,Amy,3.9,1000
123,Berkeley,CS,Y,123,Amy,3.9,1000
123,Cornell,EE,Y,123,Amy,3.9,1000
234,Berkeley,biology,N,234,Bob,3.6,1500
345,MIT,bioengineering,Y,345,Craig,3.5,500
345,Cornell,bioengineering,N,345,Craig,3.5,500
345,Cornell,CS,Y,345,Craig,3.5,500
345,Cornell,EE,N,345,Craig,3.5,500
678,Stanford,history,Y,678,Fay,3.8,200


or, using aliases, so we don't have to write long table
names:

In [10]:
%%sql
SELECT      *
FROM        applications AS a 
CROSS JOIN  students AS s
WHERE       a.s_id = s.s_id

 * sqlite:///lect02.sqlite
Done.


s_id,c_name,major,decision,s_id_1,s_name,gpa,size_hs
123,Stanford,CS,Y,123,Amy,3.9,1000
123,Stanford,EE,N,123,Amy,3.9,1000
123,Berkeley,CS,Y,123,Amy,3.9,1000
123,Cornell,EE,Y,123,Amy,3.9,1000
234,Berkeley,biology,N,234,Bob,3.6,1500
345,MIT,bioengineering,Y,345,Craig,3.5,500
345,Cornell,bioengineering,N,345,Craig,3.5,500
345,Cornell,CS,Y,345,Craig,3.5,500
345,Cornell,EE,N,345,Craig,3.5,500
678,Stanford,history,Y,678,Fay,3.8,200


We can clean up the output using a projection:

In [11]:
%%sql
SELECT      s.s_name, a.c_name, a.major
FROM        applications AS a 
CROSS JOIN  students AS s
WHERE       a.s_id = s.s_id

 * sqlite:///lect02.sqlite
Done.


s_name,c_name,major
Amy,Berkeley,CS
Amy,Cornell,EE
Amy,Stanford,CS
Amy,Stanford,EE
Bob,Berkeley,biology
Craig,Cornell,CS
Craig,Cornell,EE
Craig,Cornell,bioengineering
Craig,MIT,bioengineering
Craig,MIT,CS


or even

In [12]:
%%sql
SELECT      s_name AS name, c_name AS college, major
FROM        applications AS a
CROSS JOIN  students AS s
WHERE       a.s_id = s.s_id

 * sqlite:///lect02.sqlite
Done.


name,college,major
Amy,Berkeley,CS
Amy,Cornell,EE
Amy,Stanford,CS
Amy,Stanford,EE
Bob,Berkeley,biology
Craig,Cornell,CS
Craig,Cornell,EE
Craig,Cornell,bioengineering
Craig,MIT,bioengineering
Craig,MIT,CS


(we only need to prefix column names when there would
otherwise be an ambiguity).

There is a shortcut for cross joins, we can just put a comma
between the two tables we want to join:

In [13]:
%%sql
SELECT s_name AS name, c_name AS college, major
FROM   applications AS a, students AS s
WHERE  a.s_id = s.s_id

 * sqlite:///lect02.sqlite
Done.


name,college,major
Amy,Berkeley,CS
Amy,Cornell,EE
Amy,Stanford,CS
Amy,Stanford,EE
Bob,Berkeley,biology
Craig,Cornell,CS
Craig,Cornell,EE
Craig,Cornell,bioengineering
Craig,MIT,bioengineering
Craig,MIT,CS


This works, but there are much more elegant ways to write
this query.

An _inner join_ combine two tables into a new table (it's
not actually saved as a table) -- it does this by creating
combined rows only when rows from the two tables 'match'
each other. In this case it's the `s_id` columns which
should match, so we write:

In [14]:
%%sql
SELECT      s_name AS name, c_name AS college, major
FROM        applications AS a 
INNER JOIN  students AS s 
ON          a.s_id = s.s_id

 * sqlite:///lect02.sqlite
Done.


name,college,major
Amy,Berkeley,CS
Amy,Cornell,EE
Amy,Stanford,CS
Amy,Stanford,EE
Bob,Berkeley,biology
Craig,Cornell,CS
Craig,Cornell,EE
Craig,Cornell,bioengineering
Craig,MIT,bioengineering
Craig,MIT,CS


Logically it can be seen as the equivalent of a cross join
and then a selection, but the database engine normally use
algorithms which are much faster and requires much less
memory when we define an inner join. Once you get used to
them, inner joins will feel much better to use than cross
joins (although, in theory, the database should try to
optimize all queries, and _might_ find an efficient way to
execute even cross joins).

Inner joins are the default joins, so we can omit the word
`INNER`, and just write `JOIN`:

In [15]:
%%sql
SELECT  s_name AS name, c_name AS college, major
FROM    applications AS a
JOIN    students AS s
ON      a.s_id = s.s_id

 * sqlite:///lect02.sqlite
Done.


name,college,major
Amy,Berkeley,CS
Amy,Cornell,EE
Amy,Stanford,CS
Amy,Stanford,EE
Bob,Berkeley,biology
Craig,Cornell,CS
Craig,Cornell,EE
Craig,Cornell,bioengineering
Craig,MIT,bioengineering
Craig,MIT,CS


The condition on which we join tables is often called the
_join predicate_, and a join where the join predicate is an
equality test, such as here, is sometimes called an
_equi-join_. There is a special form of join when the
columns we're comparing in an equi-join have the same name:

In [ ]:
%%sql
SELECT  s_name AS name, c_name AS college, major
FROM    applications
JOIN    students
USING   (s_id)

One benefit from using `USING` is that the columns we join
over will not be duplicated, we get only one of them in the
output. 

I'd say this is the preferred way of writing this query.

There is an even simpler way to write it, using what's
called a _natural join_, but it's error-prone, and you're
recommended not to use it. It's written as

In [16]:
%%sql
SELECT        s_name AS name, c_name AS college, major
FROM          applications
NATURAL JOIN  students

 * sqlite:///lect02.sqlite
Done.


name,college,major
Amy,Berkeley,CS
Amy,Cornell,EE
Amy,Stanford,CS
Amy,Stanford,EE
Bob,Berkeley,biology
Craig,Cornell,CS
Craig,Cornell,EE
Craig,Cornell,bioengineering
Craig,MIT,bioengineering
Craig,MIT,CS


and it joins the tables using an equi-join for all columns
with coinciding names -- _this could create big problems if
there are attributes in the tables which just happen to
coincide without us realizing it!_ If we use the `JOIN` ...
`USING` instead, we explicitly declare on which attributes
to join, so we guard ourselves against accidental column
name collisions.

Observe that we can use selections just as before when we
join tables. 

**Exercise:** _Write a SQL query which shows all applications as
above, but only for students applying for CS at Stanford._

In [18]:
%%sql
SELECT s_name, c_name, major
FROM students
JOIN applications
USING (s_id)
WHERE c_name = 'Stanford' AND major = 'CS'

 * sqlite:///lect02.sqlite
Done.


s_name,c_name,major
Amy,Stanford,CS
Irene,Stanford,CS
Helen,Stanford,CS


**Exercise:** _Write a SQL query which shows all applications
as above, but also shows what state the college is in._

In [23]:
%%sql
SELECT s_name, c_name, major, state
FROM students
JOIN applications
USING (s_id)
JOIN colleges
USING (c_name)

 * sqlite:///lect02.sqlite
Done.


s_name,c_name,major,state
Amy,Berkeley,CS,CA
Amy,Cornell,EE,NY
Amy,Stanford,CS,CA
Amy,Stanford,EE,CA
Bob,Berkeley,biology,CA
Craig,Cornell,CS,NY
Craig,Cornell,EE,NY
Craig,Cornell,bioengineering,NY
Craig,MIT,bioengineering,MA
Craig,MIT,CS,MA


We can join a table with itself, write a query which finds
all pairs with the same Grade Point Average (GPA):

We can do it either with a cross join:

In [24]:
%%sql
SELECT s1.s_id, s1.s_name, s1.gpa, s2.s_id, s2.s_name, s2.gpa
FROM   students AS s1, students AS s2
WHERE  s1.gpa = s2.gpa AND
       s1.s_id < s2.s_id

 * sqlite:///lect02.sqlite
Done.


s_id,s_name,gpa,s_id_1,s_name_1,gpa_1
123,Amy,3.9,654,Amy,3.9
123,Amy,3.9,456,Doris,3.9
123,Amy,3.9,876,Irene,3.9
456,Doris,3.9,654,Amy,3.9
456,Doris,3.9,876,Irene,3.9
543,Craig,3.4,789,Gary,3.4
567,Edward,2.9,765,Jay,2.9
654,Amy,3.9,876,Irene,3.9


or with an inner join:

In [25]:
%%sql
SELECT s1.s_id, s1.s_name, s2.s_id, s2.s_name, gpa
FROM   students AS s1 
JOIN   students AS s2 
USING  (gpa)
WHERE  s1.s_id < s2.s_id

 * sqlite:///lect02.sqlite
Done.


s_id,s_name,s_id_1,s_name_1,gpa
123,Amy,654,Amy,3.9
123,Amy,456,Doris,3.9
123,Amy,876,Irene,3.9
456,Doris,654,Amy,3.9
456,Doris,876,Irene,3.9
543,Craig,789,Gary,3.4
567,Edward,765,Jay,2.9
654,Amy,876,Irene,3.9


Joining a table with itself is called a _self join_.
       
**Exercise:** Show the student names, college names, and
majors for all applications to Stanford:

In [ ]:
%%sql


**Exercise:** Show the average `size_hs` for applications to
the different colleges, order by descending size:

In [ ]:
%%sql


We can apply joins in several steps, each time combining
what we've previously joined with another table (the result
of which is a new table, which can be joined over and over
again...).


**Exercise:** _Show the names of all students who have
applied for a college in California -- also show the college
and major._

In [ ]:
%%sql


In lab 1 you'll get plenty of exercise in joining.


## Outer joins

An inner join combines rows in different tables _when there
is a match in the other table_, rows with no corresponding
row in the other table will not turn up in the joined table.

With an _outer join_ we can make sure that every row in one
or both of the tables turn up in the joined table -- in case
there is no match, it will be paired with a row containing
unly `NULL` values (`NULL` is written as `None` in our
notebooks).

As as example, we've seen how to join students and their
applications:

<!---
SELECT s_name, s_id, c_name, major
FROM   students
JOIN   applications
USING  (s_id)
-->

In [ ]:
%%sql


Now, assume we also want to see those students who haven't
applied, we can do that using a _left outer join_:

<!---
SELECT          s_name, s_id, c_name, major
FROM            students
LEFT OUTER JOIN applications
USING           (s_id)
-->

In [ ]:
%%sql


We still get the rows we got before, but now we also get
rows with `NULL`'s in the attributes from the right
relation, if there is no row in the right table which
corresponds to a row in the left table (i.e., the `s_id`
found in the `students` table has no match in the
`applications` table -- the tuples on the left are sometimes
called _dangling tuples_).

By the way, we can omit the `OUTER` keyword:

<!---
SELECT    s_name, s_id, c_name, major
FROM      students
LEFT JOIN applications
USING     (s_id)
-->

In [ ]:
%%sql


There is a 'natural' version of outer joins ...

<!---
SELECT            s_name, s_id, c_name, major
FROM              students
NATURAL LEFT JOIN applications
-->

In [ ]:
%%sql


... but for the same reasons as for inner joins, it's
_much_, _much_ better to explicitly declare which attribute
we're joining (using `USING`).

There is a way to get the same result without inner or outer
joins, but it requires more code:

In [ ]:
%%sql
SELECT      s_name, s.s_id, c_name, major
FROM        students s
CROSS JOIN  applications a
WHERE       s.s_id = a.s_id
UNION
SELECT      s_name, s_id, NULL, NULL
FROM        students
WHERE       s_id NOT IN (SELECT s_id
                         FROM   applications)

Seing this might help you understand what left outer join
actually returns.

Now, the opposite problem, we want to see applications with
no matching students -- of course we could just swap
`students` and `applications` in the query above, but we
could also use a _right outer join_:

In [ ]:
%%sql
SELECT     s_name, s_id, c_name, major
FROM       students
RIGHT JOIN applications USING (s_id)

Right outer joins are not supported in SQLite (yet), though.

There is also a _full outer join_, which combines the left-
and the right outer joins:

In [ ]:
%%sql
SELECT    s_name, s_id, c_name, major
FROM      students
FULL JOIN applications USING (s_id)

Of the outer joins, SQLite only supports left outer, the
other ones are left out :-).

**Exercise:** _Find all 'spurious' applications, i.e.,
applications where the student id doesn't match any
student's id._ Try to solve this exercise in two ways (one
using an outer join, the other using subqueries).

In [ ]:
%%sql


In [ ]:
%%sql


**Exercise:** _Generate the unwieldly college application
table we started out with above from the three smaller
tables._

In [ ]:
%%sql


## Back to the olympics:

Last time we had a few queries using our old `olympics`
table, let's revisit one of them using `better_olympics` and
`continents` instead. First we rename our `olympics` and
`better_olympics` tables, to make things easier:

In [ ]:
%%sql
ALTER TABLE olympics
RENAME TO   bad_olympics;

ALTER TABLE better_olympics
RENAME TO   olympics;

**Q:** _How many olympic games has each continent hosted?_

In [ ]:
%%sql


**Q:** _When was the first and the latest olympic games in
each continent?_

In [ ]:
%%sql


**Q:** _Recreate the original `olympics` table from the
improved `olympics` and `continents`._

In [ ]:
%%sql


## Instead of lists

One of the 'rules' of relational databases is that the
values in our tables must be primitive, we're not allowed to
have lists or objects as values. This requirement is called
the ["First normal
form"](https://en.wikipedia.org/wiki/First_normal_form), or
"1NF" -- we'll return to 'normal forms' during lectures 7
and 8.

If we were to write a program to keep track of the phone
numbers of our friends, we could write something like this
in Java:

~~~java
class Friend {

    private String name;
    private List<PhoneNumber> phoneNumbers;
    
    ...
}

List<Friend> friends = ...
~~~


A literal translation of this into a table would be
something like:

~~~
name      phone_numbers
----      -------------
Adam      [123456, 654321]
Bodil     [196811]
~~~


but this is in violation of the 'First normal form', so
instead have a new row for each phone number:

~~~
name      phone
----      -----
Adam      123456
Adam      654321
Bodil     196811
~~~


We can ask for Adam's number with:

~~~sql
SELECT phone
FROM   friends
WHERE  name = 'Adam'
~~~


and we'd get both numbers back:

~~~
phone
-----
123456
654321
~~~


Now, assume we also want to save the birthdays of our
friends. We could add a column for birthdays like this:

~~~
name    birthday    phone
----    --------    -----
Adam    2 dec       123456
Adam    2 dec       654321
Bodil   30 nov      196811
~~~


What's the problem with this?

And what if we also want to save our friends' email
addresses:

~~~
name        birthday    phone       email
----        --------    -----       -----
Adam        2 dec       123456      adam@gmail.com
Adam        2 dec       123456      adam@yahoo.com
Adam        2 dec       654321      adam@gmail.com
Adam        2 dec       654321      adam@yahoo.com
Bodil       30 nov      196811      bodil@itu.dk
Bodil       30 nov      196811      bodil@lu.se
Cecilia     9 apr       511235      cecilia@dn.se
Cecilia     9 apr       641587      cecilia@dn.se
David       12 jun      984531      david@gmail.co
David       12 jun      984531      david@hotmail
Emma        11 aug      123456      emma@lu.se
~~~


It may seem silly to have four rows for Adam, but if we
removed one of them, we would loose vital information. If we
removed the first row, and someone asked who had the phone
number 123456 and the email address adam@gmail.com, the
query:

~~~sql
SELECT  name, birthday
FROM    friends
WHERE   phone = '123456' AND email = 'adam@gmail.com'
~~~


would give an empty result.

The problem with the table above is that we have a lot of
redundance, i.e., information is repeated in many places.
This is not only a waste of storage space, it also makes it
much, much harder to work with our table -- insertions,
updates and removals affects many rows, and if we're not
careful, we might leave the table in a corrupted state.

**Exercise:** _The data in the table above would be much
easier to maintain if we split it into several tables --
suggest how to do that._


## Views

If we often use some query as part of other queries, we can
save it using a _view_:

In [ ]:
%%sql
CREATE VIEW application_info AS
  SELECT s_id, s_name AS name, major, c_name AS college, state
  FROM   applications
  JOIN   students
  USING  (s_id)
  JOIN   colleges
  USING  (c_name);

SELECT DISTINCT name, state
FROM   application_info
WHERE  s_id IN (123, 234, 456);

**Exercise:** _Show the year and category for everyone who
has won more than one Nobel Prize -- use a view to do it._

<!---
CREATE VIEW multiple_awards AS
  SELECT name
  FROM   nobel
  GROUP BY laureate
  HAVING COUNT() > 1;

SELECT year, category, name
FROM   multiple_awards
JOIN   nobel
USING  (name)
ORDER BY year;
-->

We first create a view with the names of laureates with
multiple awards:

In [ ]:
%%sql


and then use it to collect information about each of the
awards:

In [ ]:
%%sql


A view will be around until someone explicitly drops it:

In [ ]:
%%sql
DROP VIEW IF EXISTS multiple_awards;

## The `WITH`-statement

As an alternative to views, we have _Common Table
Expressions_ (or _CTE_'s) -- they are a kind of named view
which is only used in a single query:

In [ ]:
%%sql
WITH multiple_awards AS
  (SELECT name
   FROM   nobel
   GROUP BY name
   HAVING COUNT() > 1)
SELECT year, category, name
FROM   multiple_awards
JOIN   nobel
USING  (name)
ORDER BY year;

Unlike views, tables created with `WITH` will 'disappear' as
soon as our query is finished, one point of the `WITH`
statement is that it's part of one single query (whereas the
creation of a view is a statement itself) -- this will
become useful when we make calls to our database from other
programs (in lecture 7 and 8).


We can't have another `WHERE` after the `HAVING` clause:

In [ ]:
%%sql
SELECT    category, count() AS cnt
FROM      nobel
WHERE     year = 2013
GROUP BY  category
HAVING    count() < 3
WHERE     cnt > 1        -- not allowed!

but we can make our 'grouping query' into a subquery, and
have another `WHERE` in the outer query

In [ ]:
%%sql
SELECT category, cnt
FROM (
    SELECT    category, count() AS cnt
    FROM      nobel
    WHERE     year = 2013
    GROUP BY  category
    HAVING    count() < 3)
WHERE cnt > 1

A somewhat tidier way of expressing this is to use a
`WITH`-statement -- we can write either:

In [ ]:
%%sql
WITH category_count AS (
    SELECT    category, count() AS cnt
    FROM      nobel
    WHERE     year = 2013
    GROUP BY  category
    HAVING    count() < 3
)
SELECT category, cnt
FROM   category_count
WHERE  cnt > 1

or

In [ ]:
%%sql
WITH category_count(category, cnt) AS (
    SELECT    category, count()
    FROM      nobel
    WHERE     year = 2013
    GROUP BY  category
    HAVING    count() < 3
)
SELECT category, cnt
FROM   category_count
WHERE  cnt > 1